This notebook is to evaluate pulling photos for places using the google places API.
nearby search: https://developers.google.com/maps/documentation/places/web-service/nearby-search#fieldmask 

Some other fields that can be obtained from this API:

The following fields trigger the Nearby Search (Basic) SKU:

places.accessibilityOptions, places.addressComponents, places.adrFormatAddress, places.businessStatus, places.displayName, places.formattedAddress, places.googleMapsUri, places.iconBackgroundColor, places.iconMaskBaseUri, places.id, places.location, places.name*, places.photos, places.plusCode, places.primaryType, places.primaryTypeDisplayName, places.shortFormattedAddress, places.subDestinations, places.types, places.utcOffsetMinutes, places.viewport

* The places.name field contains the place resource name in the form: places/PLACE_ID. Use places.displayName to access the text name of the place.

The following fields trigger the Nearby Search (Advanced) SKU:

places.currentOpeningHours, places.currentSecondaryOpeningHours, places.internationalPhoneNumber, places.nationalPhoneNumber, places.priceLevel, places.rating, places.regularOpeningHours, places.regularSecondaryOpeningHours, places.userRatingCount, places.websiteUri

The following fields trigger the Nearby Search (Preferred) SKU:

places.allowsDogs, places.curbsidePickup, places.delivery, places.dineIn, places.editorialSummary, places.evChargeOptions, places.fuelOptions, places.goodForChildren, places.goodForGroups, places.goodForWatchingSports, places.liveMusic, places.menuForChildren, places.parkingOptions, places.paymentOptions, places.outdoorSeating, places.reservable, places.restroom, places.reviews, places.servesBeer, places.servesBreakfast, places.servesBrunch, places.servesCocktails, places.servesCoffee, places.servesDesserts, places.servesDinner, places.servesLunch, places.servesVegetarianFood, places.servesWine, places.takeout

In [52]:
from dotenv import load_dotenv
import requests
import os
import pandas as pd
import math

In [76]:
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")
API_URL = "https://places.googleapis.com/v1/places:searchText"
PHOTO_API_URL = "https://places.googleapis.com/v1"

In [78]:
osm_places = pd.read_csv("../osm_places/osm_places_amenity.csv")
osm_places.head()

C:\Users\Daniel\AppData\Local\Temp\ipykernel_23496\469650520.py:1: DtypeWarning: Columns (47,70) have mixed types. Specify dtype option on import or set low_memory=False.
  osm_places = pd.read_csv("../osm_places/osm_places_amenity.csv")


,id,lat,lon,addr:city,addr:housenumber,addr:postcode,addr:state,addr:street,amenity,branch,...,addr:unit,contact:website,short_name,indoor_seating,drink:sake,contact:email,diet:kosher,diet:halal,contact:phone,name:zh
0,349323821,40.762515,-73.976690,New York,43,10019,NY,West 55th Street,fast_food,Manhattan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,357618253,40.740822,-73.784295,Fresh Meadows,61-09,11365,NY,190th Street,restaurant,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,357620442,40.741558,-73.978463,NaN,NaN,NaN,NaN,NaN,restaurant,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,357621192,40.846563,-73.938127,New York,4140,10033,NY,Broadway,theatre,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,368042980,40.721480,-73.997103,NaN,97,10012,NaN,Kenmare Street,arts_centre,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
osm_places.columns.values

array(['id', 'lat', 'lon', 'addr:city', 'addr:housenumber',
       'addr:postcode', 'addr:state', 'addr:street', 'amenity', 'branch',
       'brand', 'brand:wikidata', 'cuisine', 'drive_through', 'name',
       'official_name', 'opening_hours', 'phone', 'takeaway', 'website',
       'outdoor_seating', 'cocktails', 'drink:beer', 'drink:liquor',
       'drink:wine', 'air_conditioning', 'wikidata', 'wikipedia',
       'source', 'wheelchair', 'addr:country', 'sport', 'check_date',
       'payment:american_express', 'payment:cash', 'payment:contactless',
       'payment:credit_cards', 'payment:debit_cards',
       'payment:mastercard', 'payment:visa', 'start_date', 'email', 'fax',
       'description', 'name:en', 'operator', 'alt_name', 'ref',
       'internet_access', 'internet_access:fee', 'opening_hours:signed',
       'level', 'atm', 'opening_hours:kitchen', 'brewery', 'diet:vegan',
       'diet:vegetarian', 'toilets', 'toilets:unisex', 'drink:coffee',
       'drink:tea', 'shop', 'smoki

In [80]:
for index, row in osm_places.loc[0:10].iterrows():
    print(row['id'])

349323821
357618253
357620442
357621192
368042980
368043384
368043485
368043598
368043799
368043902
368045827


In [87]:
df = pd.read_csv("google_places.csv")

In [89]:
header = {
    "Content-Type": "v",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "places.displayName,places.name,places.location,places.photos"
}


places_data = [] #List of dictionaries for conversion into pd dataframe

for index, row in osm_places.loc[0:300].iterrows():
    # Check dataframe if place has already been queried, if so skip to save API calls
    if row['id'] in df['id'].values:
        print('skipping', row['id'])
        continue

    req = {
        "textQuery": row['name'],
        "pageSize": 3,
        "locationBias": {
            "circle": {
                "center": {
                    "latitude": row['lat'], 
                    "longitude": row['lon']
                },
                "radius": 200.0
            }
        }
    }
    # API request
    try:
        response = requests.post(API_URL, json=req, headers=header)
        response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print("HTTP Error")
        print(errh.args[0])
    except requests.exceptions.RequestException as e:
        print("Request Exception:", e)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data from the response
        data = response.json()        
        print(data)
        
        # If response has no places, add row ID to places data so we don't query again and continue.
        if 'places' not in data.keys():
            places_data.append({'id':row['id']})
            continue

        # Just interested in the closest place
        place = data['places'][0]

        place_data = {'id':row['id'], 
                      'google_name': place['name'], 
                      'disp_name': place['displayName']['text'], 
                      'lat': place['location']['latitude'],
                      'lon':place['location']['longitude']}
        # Add photos
        if 'photos' in place.keys():
            for i, photo in enumerate(place['photos']):
                place_data[f'photo_{i}'] = photo['name']
        
        places_data.append(place_data)
        print(place_data)
    else:
        # Print the status code if the request failed
        print(f'Failed to retrieve data: {response.status_code}')
        print(response)



skipping 349323821
skipping 357618253
skipping 357620442
skipping 357621192
skipping 368042980
skipping 368043384
skipping 368043485
skipping 368043598
skipping 368043799
skipping 368043902
skipping 368045827
skipping 368045830
skipping 368045855
skipping 368045862
skipping 368045872
skipping 368045890
skipping 368045910
skipping 368047348
skipping 368049242
skipping 368049266
skipping 368049376
skipping 368049383
skipping 368049415
skipping 368049421
skipping 368049705
skipping 368049888
skipping 368049984
skipping 368050015
skipping 368050018
skipping 368050028
skipping 368050133
skipping 368050526
skipping 368050559
skipping 368050602
skipping 368050625
skipping 368050633
skipping 368050642
skipping 368050654
skipping 368051450
skipping 368051456
skipping 368051467
skipping 368051474
skipping 368051477
skipping 368051495
skipping 368051619
skipping 368051660
skipping 368051665
skipping 368051668
skipping 368051674
skipping 368051679
skipping 368051684
skipping 368051690
skipping 368

In [90]:
new_df = pd.DataFrame(places_data)
new_df

,id,google_name,disp_name,lat,lon,photo_0,photo_1,photo_2,photo_3,photo_4,photo_5,photo_6,photo_7,photo_8,photo_9
0,561239802,places/ChIJNWlUIgRZwokRUkQz7u2ROdo,Delectica,40.748153,-73.976687,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...,places/ChIJNWlUIgRZwokRUkQz7u2ROdo/photos/AUGG...
1,566313758,places/ChIJy1euYgPzwokRh0HmWpSAF08,Inwood Farm,40.872778,-73.918349,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...,places/ChIJy1euYgPzwokRh0HmWpSAF08/photos/AUGG...
2,566316071,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk,Garden Cafe,40.868165,-73.920528,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...,places/ChIJ6TCT6PbzwokRfsZUbEWFNWk/photos/AUGG...
3,567809178,places/ChIJKUSJs9z0wokRtCxjjToNE2Q,Taco Bell,40.826375,-73.860656,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...,places/ChIJKUSJs9z0wokRtCxjjToNE2Q/photos/AUGG...
4,570912397,places/ChIJqRJTizv2wokRwg_FF-6pYuU,Le Monde,40.805886,-73.965885,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...,places/ChIJqRJTizv2wokRwg_FF-6pYuU/photos/AUGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1427499791,places/ChIJf9GUrp5ZwokRcRxMvzOronY,Heavenly Market and Deli,40.731441,-73.988241,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...
196,1427499828,places/ChIJjeTT_DxXwokR1sB-qcPZO8k,Fuel 4 Food Mart,40.743989,-74.063014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,1427499831,places/ChIJj7cR5gBZwokRXylE1glTthU,Chipotle Mexican Grill,40.751360,-73.980721,places/ChIJj7cR5gBZwokRXylE1glTthU/photos/AUGG...,places/ChIJj7cR5gBZwokRXylE1glTthU/photos/AUGG...,places/ChIJ

In [91]:
df = pd.concat([df, new_df])
df

,id,google_name,disp_name,lat,lon,photo_0,photo_1,photo_2,photo_3,photo_4,photo_5,photo_6,photo_7,photo_8,photo_9
0,349323821,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc,Five Guys,40.762546,-73.976704,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...
1,357618253,places/ChIJcdatubBhwokRNd0ySdodDgg,Hooters of Fresh Meadows,40.740789,-73.784351,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...
2,357620442,places/ChIJG8WWttZZwokRBkeFkHkoFv4,Sam Sunny,40.741594,-73.978530,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...
3,357621192,places/ChIJ00nIlp_2wokROYs-HVvAfj8,United Palace,40.846539,-73.937923,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...
4,368042980,places/ChIJeaNd3IhZwokRU25epwuJ4aY,Storefront for Art and Architecture,40.721478,-73.997071,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1427499791,places/ChIJf9GUrp5ZwokRcRxMvzOronY,Heavenly Market and Deli,40.731441,-73.988241,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...
196,1427499828,places/ChIJjeTT_DxXwokR1sB-qcPZO8k,Fuel 4 Food Mart,40.743989,-74.063014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,1427499831,places/ChIJj7cR5gBZwokRXylE1glTthU,Chipotle Mexican Grill,40.751360,-73.980721,places/ChIJj7cR5gBZwokRXylE1glTthU/photos/AUGG...,places/ChIJj7cR5gBZ

In [92]:
df = df.drop_duplicates(subset=['id'])
df

,id,google_name,disp_name,lat,lon,photo_0,photo_1,photo_2,photo_3,photo_4,photo_5,photo_6,photo_7,photo_8,photo_9
0,349323821,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc,Five Guys,40.762546,-73.976704,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...,places/ChIJ_b6qJvpYwokR7TFSUhdqGUc/photos/AUGG...
1,357618253,places/ChIJcdatubBhwokRNd0ySdodDgg,Hooters of Fresh Meadows,40.740789,-73.784351,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...,places/ChIJcdatubBhwokRNd0ySdodDgg/photos/AUGG...
2,357620442,places/ChIJG8WWttZZwokRBkeFkHkoFv4,Sam Sunny,40.741594,-73.978530,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...,places/ChIJG8WWttZZwokRBkeFkHkoFv4/photos/AUGG...
3,357621192,places/ChIJ00nIlp_2wokROYs-HVvAfj8,United Palace,40.846539,-73.937923,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...,places/ChIJ00nIlp_2wokROYs-HVvAfj8/photos/AUGG...
4,368042980,places/ChIJeaNd3IhZwokRU25epwuJ4aY,Storefront for Art and Architecture,40.721478,-73.997071,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...,places/ChIJeaNd3IhZwokRU25epwuJ4aY/photos/AUGG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1427499791,places/ChIJf9GUrp5ZwokRcRxMvzOronY,Heavenly Market and Deli,40.731441,-73.988241,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...,places/ChIJf9GUrp5ZwokRcRxMvzOronY/photos/AUGG...
196,1427499828,places/ChIJjeTT_DxXwokR1sB-qcPZO8k,Fuel 4 Food Mart,40.743989,-74.063014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,1427499831,places/ChIJj7cR5gBZwokRXylE1glTthU,Chipotle Mexican Grill,40.751360,-73.980721,places/ChIJj7cR5gBZwokRXylE1glTthU/photos/AUGG...,places/ChIJj7cR5gBZ

In [93]:
df.to_csv("google_places.csv", index=False)